In [1]:
from dotenv import load_dotenv
import os

from langchain_community.graphs import Neo4jGraph

# Warning control
import warnings
warnings.filterwarnings("ignore")

/home/ravi0531rp/anaconda3/envs/langchain-env/lib/python3.11/site-packages/langchain_community/graphs/neo4j_graph.py:6: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_community.graphs.graph_document import GraphDocument


In [2]:
load_dotenv('.env', override=True)
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
NEO4J_DATABASE = os.getenv('NEO4J_DATABASE')

In [3]:
kg = Neo4jGraph(
    url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE
)

In [4]:
kg

In [5]:
cypher = """
  MATCH (n) 
  RETURN count(n)
  """

In [6]:
result = kg.query(cypher)
result

[{'count(n)': 131}]

In [7]:
cypher = """
  MATCH (n) 
  RETURN count(n) AS numberOfNodes
  """

In [8]:
result = kg.query(cypher)
result

[{'numberOfNodes': 131}]

In [9]:
cypher = """
  MATCH (m:Movie) 
  RETURN count(m) AS numberOfMovies
  """
kg.query(cypher)

[{'numberOfMovies': 100}]

In [10]:
cypher = """
    MATCH (p: Person)
    RETURN count(p) AS numberOfPeople
"""
kg.query(cypher)

[{'numberOfPeople': 31}]

In [11]:
cypher = """
  MATCH (Person_2:Person {name:"Person_2"}) 
  RETURN Person_2
  """
kg.query(cypher)

[{'Person_2': {'born': '1940-02-13', 'name': 'Person_2'}},
 {'Person_2': {'name': 'Person_2'}}]

In [12]:
cypher = """
  MATCH (mv:Movie {title:"Movie_20"}) 
  RETURN mv
  """
kg.query(cypher)

[{'mv': {'tagline': 'Tagline for Movie_20',
   'title': 'Movie_20',
   'released': '2008-02-24'}}]

### Conditional Matching

In [13]:
cypher = """
  MATCH (nineties:Movie) 
WHERE nineties.released < '1999-01-01'
  RETURN nineties
  """
kg.query(cypher)

[{'nineties': {'tagline': 'Tagline for Movie_5',
   'title': 'Movie_5',
   'released': '1998-03-26'}},
 {'nineties': {'tagline': 'Tagline for Movie_7',
   'title': 'Movie_7',
   'released': '1996-04-20'}},
 {'nineties': {'tagline': 'Tagline for Movie_10',
   'title': 'Movie_10',
   'released': '1985-01-10'}},
 {'nineties': {'tagline': 'Tagline for Movie_11',
   'title': 'Movie_11',
   'released': '1980-02-28'}},
 {'nineties': {'tagline': 'Tagline for Movie_14',
   'title': 'Movie_14',
   'released': '1981-05-05'}},
 {'nineties': {'tagline': 'Tagline for Movie_24',
   'title': 'Movie_24',
   'released': '1996-05-25'}},
 {'nineties': {'tagline': 'Tagline for Movie_25',
   'title': 'Movie_25',
   'released': '1995-12-13'}},
 {'nineties': {'tagline': 'Tagline for Movie_30',
   'title': 'Movie_30',
   'released': '1990-12-19'}},
 {'nineties': {'tagline': 'Tagline for Movie_34',
   'title': 'Movie_34',
   'released': '1992-03-12'}},
 {'nineties': {'tagline': 'Tagline for Movie_39',
   'title

### Pattern Matching with Multiple nodes

In [14]:
cypher = """
  MATCH (actor:Person {name : "Person_1"})-[:ACTED_IN]->(movie:Movie) 
  RETURN actor.name, movie.title LIMIT 10
  """
kg.query(cypher)

[{'actor.name': 'Person_1', 'movie.title': 'Movie_28'},
 {'actor.name': 'Person_1', 'movie.title': 'Movie_90'}]

In [15]:
cypher = """
  MATCH (directors:Person)-[:DIRECTED]->(movie:Movie) 
  RETURN directors.name, movie.title LIMIT 10
  """
kg.query(cypher)

[{'directors.name': 'Person_2', 'movie.title': 'Movie_80'},
 {'directors.name': 'Person_2', 'movie.title': 'Movie_88'},
 {'directors.name': 'Person_2', 'movie.title': 'Movie_76'},
 {'directors.name': 'Person_3', 'movie.title': 'Movie_46'},
 {'directors.name': 'Person_3', 'movie.title': 'Movie_21'},
 {'directors.name': 'Person_3', 'movie.title': 'Movie_50'},
 {'directors.name': 'Person_3', 'movie.title': 'Movie_5'},
 {'directors.name': 'Person_5', 'movie.title': 'Movie_86'},
 {'directors.name': 'Person_5', 'movie.title': 'Movie_58'},
 {'directors.name': 'Person_6', 'movie.title': 'Movie_77'}]

In [16]:
cypher = """
  MATCH (actor:Person)-[:DIRECTED]->(movie:Movie) 
  RETURN count(actor : Person) AS numberOfDirectors
  """
kg.query(cypher)

[{'numberOfDirectors': 26}]

In [17]:
cypher = """
  MATCH (tom:Person {name:"Person_5"})-[:ACTED_IN]->(m)<-[:ACTED_IN]-(coActors) 
  RETURN coActors.name, m.title
  """
kg.query(cypher)

[]

### Delete

In [21]:
cypher = """
MATCH (emil:Person {name:"Person_1"})-[actedIn:ACTED_IN]->(movie:Movie)
RETURN emil.name, movie.title
"""
kg.query(cypher)

[{'emil.name': 'Person_1', 'movie.title': 'Movie_28'},
 {'emil.name': 'Person_1', 'movie.title': 'Movie_90'}]

In [22]:
cypher = """
MATCH (emil:Person {name:"Person_1"})-[actedIn:ACTED_IN]->(movie:Movie)
DELETE actedIn
"""
kg.query(cypher)

[]

### Add

In [23]:
cypher = """
CREATE (andreas:Person {name:"Person_88888"})
RETURN andreas
"""

kg.query(cypher)

[{'andreas': {'name': 'Person_88888'}}]

In [24]:
cypher = """
MATCH (andreas:Person {name:"Person_88888"}), (emil:Person {name:"Person_10"})
MERGE (andreas)-[hasRelationship:WORKS_WITH]->(emil)
RETURN andreas, hasRelationship, emil
"""
kg.query(cypher)

[{'andreas': {'name': 'Person_88888'},
  'hasRelationship': ({'name': 'Person_88888'},
   'WORKS_WITH',
   {'born': '1969-07-28', 'name': 'Person_10'}),
  'emil': {'born': '1969-07-28', 'name': 'Person_10'}}]

### Prepare Text Data for RAG

In [29]:
kg.query("""
  CREATE VECTOR INDEX movie_tagline_embeddings IF NOT EXISTS
  FOR (m:Movie) ON (m.taglineEmbedding) 
  OPTIONS { indexConfig: {
    `vector.dimensions`: 768,
    `vector.similarity_function`: 'cosine'
  }}"""
)


[]

In [30]:
kg.query("""
  SHOW VECTOR INDEXES
  """
)

[{'id': 3,
  'name': 'movie_tagline_embeddings',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'VECTOR',
  'entityType': 'NODE',
  'labelsOrTypes': ['Movie'],
  'properties': ['taglineEmbedding'],
  'indexProvider': 'vector-2.0',
  'owningConstraint': None,
  'lastRead': None,
  'readCount': 0}]

In [31]:
import google.generativeai as genai
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))


In [ ]:
kg.query("""
    MATCH (movie:Movie) WHERE movie.tagline IS NOT NULL
    WITH movie, genai.vector.encode(
        movie.tagline, 
        "OpenAI", 
        {
          token: $openAiApiKey,
          endpoint: $openAiEndpoint
        }) AS vector
    CALL db.create.setNodeVectorProperty(movie, "taglineEmbedding", vector)
    """, 
    params={"openAiApiKey":OPENAI_API_KEY, "openAiEndpoint": OPENAI_ENDPOINT} )

In [ ]:
question = "What movies are about love?"

In [ ]:
kg.query("""
    WITH genai.vector.encode(
        $question, 
        "OpenAI", 
        {
          token: $openAiApiKey,
          endpoint: $openAiEndpoint
        }) AS question_embedding
    CALL db.index.vector.queryNodes(
        'movie_tagline_embeddings', 
        $top_k, 
        question_embedding
        ) YIELD node AS movie, score
    RETURN movie.title, movie.tagline, score
    """, 
    params={"openAiApiKey":OPENAI_API_KEY,
            "openAiEndpoint": OPENAI_ENDPOINT,
            "question": question,
            "top_k": 5
            })